# Self-study try-it activity 4.3: Bootstrapping in Python

Bootstrapping is a resampling technique used to create multiple training data sets by sampling the original data set with replacement. Each bootstrapped data set is of the same size as the original, but it may include duplicate observations while leaving others out. This method is widely used to estimate model stability, reduce overfitting and improve robustness.

## How bootstrapping works:

- Sampling with replacement: randomly draw data points from the original data set, allowing duplicates.

- Training multiple models: use each bootstrapped sample to train a separate model.

- Aggregating predictions: combine predictions from all models, typically through averaging (for regression) or majority voting (for classification), to improve accuracy and reduce variance.

In [ ]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from scipy import optimize
from scipy.stats import multivariate_normal
np.random.seed(1)

Generate a data set of 1,000 students with a mean height of 170 and standard deviation of 10. Perform bootstrapping and compute the new means and the `confidence_interval`, assuming a 95 per cent confidence.

In [ ]:
#Generate a data set (e.g. heights of students)
data = np.random.normal(loc=170, scale=10, size=1000)  #Mean = 170, StdDev = 10, size = 1000

#Number of bootstrap samples
n_bootstrap_samples = 1000

#Store means of bootstrap samples
bootstrap_means = []

#Perform bootstrapping
for _ in range(n_bootstrap_samples):
    #Sample with replacement from the original data
    bootstrap_sample = np.random.choice(data, size=len(data), replace=True)
    #Calculate the mean of the bootstrap sample
    bootstrap_means.append(np.mean(bootstrap_sample))

#Calculate the overall mean and confidence interval
mean_estimate = np.mean(bootstrap_means)
confidence_interval = (np.percentile(bootstrap_means, 2.5), np.percentile(bootstrap_means, 97.5))

print(f"Estimated Mean: {mean_estimate}")
print(f"95% Confidence Interval: {confidence_interval}")


## Examples of bootstrapping

These examples are inspired by James, Gareth, Daniela Witten, Trevor Hastie, and Rob Tibshirani. *An introduction to statistical learning.* Springer, 2013.

### Motivation

Let's review two different stocks: $X$ and $Y$, normally distributed with the same mean of $\mu_X := \mathbb{E}[X] = 2$ and $\mu_Y := \mathbb{E}[Y] = 2$. 

Additionally, $\sigma^2_X := \mathbb{V}\mathrm{AR}[X] = 1$, $\sigma^2_Y := \mathbb{V}\mathrm{AR}[Y] = 1.25$, and $\sigma_{XY} := \mathbb{C}\mathrm{OV}[X,Y] = 0.5$.

### Question 1: How can you sample $1,000$ returns from these stocks?

**Answer 1:** The function `np.random.multivariate_normal(mu, r, size=n)` generates random samples from a multivariate normal (Gaussian) distribution.

In [ ]:
n = 1000

#Vector of means
mu = np.array([2 ,2])

#Describe covariance matrix
r = np.array([
        [  1,    0.5],
        [ 0.5,  1.25]
    ])

#Generate the random samples
returns = np.random.multivariate_normal(mu, r, size=n)
print(returns)

In [ ]:
# Plot the returns
x = returns[:,0]
y = returns[:,1]
x1, y1 = [-1, 5.5], [-1, 5.5]
plt.plot(x1, y1, c = "r", alpha= .3)
plt.title(r"Returns of stocks $X$ and $Y$, points below the red line are the cases where $X$ has a larger return")
plt.xlabel(r"$X$")
plt.ylabel(r"$Y$")
plt.scatter(x,y,s=2)
plt.show()

### Question 2: How can you find an optimal $\alpha^\star$?

Let's invest in two stocks: $X$ and $Y$. Allocate a portion $\alpha \in [0,1]$ of your money to stock $X$ and the remaining $1 - \alpha$ to stock $Y$. The goal is to minimise the risk of this investment, where risk is measured by the variance of the portfolio return.

In other words, you need to solve:

$\alpha^\star \in \arg\min_{\alpha} \{ \mathbb{V}\mathrm{AR}[\alpha X + (1-\alpha)Y] \}$

How can you find an optimal $\alpha^\star$?

**Answer 2:** Recall that

\begin{align*} \mathbb{V}\mathrm{AR}[\alpha X + (1-\alpha)Y] &= \alpha^2 \mathbb{V}\mathrm{AR}[X] + (1-\alpha)^2 \mathbb{V}\mathrm{AR}[Y] + 2\alpha(1-\alpha) \mathbb{C}\mathrm{OV}[X,Y]\\
& = \alpha^2 \sigma^2_X + (1-\alpha)^2 \sigma^2_Y + 2\alpha(1-\alpha) \sigma_{XY}
\end{align*}

Let's minimise this function by plugging in the true values of $\sigma_X^2 = 1,\  \sigma_Y^2 = 1.25, \ \sigma_{XY} = 0.5$, using the ```scipy optimize``` function. 

Note: `round(float(solution.x), 3)` is used. This extracts the scalar solution from the single-value array returned by the optimizer and rounds it to three decimal places for cleaner, more readable output.

When plugging these values, the problem is:

\begin{align*}
\begin{array}{ll}
\text{minimise}_\alpha & (1-\alpha)^2 \times 1.25 + \alpha \\
\text{subject to} & \alpha \in [0,1].
\end{array}
\end{align*}

In [ ]:
def objective_given(alpha):
    """The objective function to minimize"""
    return (1-alpha)**2 * 1.25 + alpha
bounds = optimize.Bounds([0],[1])
solution = optimize.minimize(objective_given, x0 = [0], bounds = bounds)

In [ ]:
print("The optimal solution is when alpha = ", round(float(solution.x),3))

What happens if you keep $\sigma^2_X, \sigma^2_Y, \sigma_{XY}$ as parameters and change them whenever you want? 

In other words, you need to solve:

\begin{align*}
\begin{array}{ll}
\text{minimise}_\alpha & \alpha^2 \sigma^2_X + (1-\alpha)^2 \sigma^2_Y + 2\alpha(1-\alpha) \sigma_{XY} \\
\text{subject to} & \alpha \in [0,1].
\end{array}
\end{align*}

In [ ]:
def objective_given(alpha, varx, vary, covar):
    """The objective function to minimize"""
    return (alpha**2 * varx) + ((1-alpha)**2 * vary) + (2*alpha*(1-alpha)*covar)
bounds = optimize.Bounds([0],[1])
solution = optimize.minimize(objective_given, x0 = [0], bounds = bounds, args=(1,1.25,0.5)) #this was previous case

In [ ]:
print("The optimal solution is when alpha = ", round(float(solution.x),3))

In [ ]:
solution = optimize.minimize(objective_given, x0 = [0], bounds = bounds, args=(5,1.25,0.5))
print("The optimal solution is when alpha = ", round(float(solution.x),3))

You can see that the larger $\sigma^2_X$ gets, the less you invest in $X$ as $\alpha$ decreases. As $\sigma^2_X$ goes up, meaning the riskier the asset, the less we invest in it to keep the portfolio balanced. 

A common mistake in optimisation practice is over-relying on solvers when, in many cases, the solution can be derived analytically. Doing so has a key advantage: the structure of the optimal solution becomes explicitly parametrised by 
$\sigma_X^2, \ \sigma_Y^2, \ \sigma_{XY}$. This not only simplifies computation but also gives deeper intuition into the investment strategy.

Let’s walk through how that works.

The objective function can be rewritten as:

\begin{align*} \mathbb{V}\mathrm{AR}[\alpha X + (1-\alpha)Y] &= \alpha^2 \sigma^2_X + (1-\alpha)^2 \sigma^2_Y + 2\alpha(1-\alpha) \sigma_{XY} \\
& = \alpha^2 \sigma^2_X + \sigma^2_Y + \alpha^2 \sigma^2 Y - 2\alpha \sigma^2_Y + 2\alpha \sigma_{XY} - 2\alpha^2 \sigma_{XY} \\
& = \alpha^2(\sigma^2_X + \sigma^2_Y - 2\sigma_{XY}) + 2\alpha(\sigma_{XY} - \sigma^2_Y) + \sigma_Y^2
\end{align*}

Notice that this is a quadratic function of $\alpha$, and the coefficient of $\alpha^2$ is $\sigma^2_X + \sigma^2_Y - 2\sigma_{XY}$, which is $\mathbb{V}\mathrm{AR}[X - Y] \geq 0$. Hence, $\alpha^2$ has a coefficient term, concluding the objective function is **convex**. So, if you take the first-order conditions, then the solution that solves them is optimal. In other words:

\begin{align*}
\dfrac{\mathrm{d}\left[ \alpha^2(\sigma^2_X + \sigma^2_Y - 2\sigma_{XY}) + 2\alpha(\sigma_{XY} - \sigma^2_Y) + \sigma_Y^2 \right]}{\mathrm{d} \alpha} = 2\alpha (\sigma^2_X + \sigma^2_Y - 2\sigma_{XY}) + 2(\sigma_{XY} - \sigma^2_Y)
\end{align*}

Optimal $\alpha^\star$ sets the above expression equal to zero; hence, it immediately follows that:

\begin{align*}
\alpha^\star = \dfrac{\sigma^2_Y - \sigma_{XY}}{\sigma^2_X + \sigma^2_Y - 2\sigma_{XY}}.
\end{align*}

Let's test whether this is correct now:

In [ ]:
def optimal_alpha(varx, vary, covar):
    return (float) (vary - covar)/(varx + vary - 2*covar)
round(optimal_alpha(1, 1.25, 0.5),3)

In [ ]:
round(optimal_alpha(5, 1.25, 0.5),3)

### Question 3: How can you decide the optimal investment strategy for the future by looking at the **past data**?


In real life, you **never** know the distribution of stocks $X$ and $Y$. Hence, you can only **estimate** $\sigma^2_X, \ \sigma^2_Y, \ \sigma_{XY}$ from a sample of $(X,Y)$ realisations. 

Let's have $n$ samples of $(X,Y)$ and name them $(X_i, Y_i)$ for $i=1,\ldots,n$. 

How can you decide the optimal investment strategy for the future by looking at the **past data**?

**Answer 3:** Use the sample provided to estimate $\sigma_X^2$, $\sigma_Y^2$, $\sigma_{XY}$. Let these estimates be $\hat{\sigma}_X^2, \ \hat{\sigma}_Y^2, \ \hat{\sigma}_{XY}$.

#### Step 1: Estimate the mean returns $\bar{X} and \ \bar{Y}$ by:

\begin{align*}
\bar{X} &= \frac{1}{n}\sum_{i=1}^n X_i \\
\bar{Y} &= \frac{1}{n}\sum_{i=1}^n Y_i
\end{align*}

In [ ]:
hat_mean = np.mean(returns,0)
print(hat_mean)

#### Step 2: Estimate the covariance matrix by:

\begin{align*}
\begin{bmatrix}
\hat{\sigma}^2_X & \hat{\sigma}_{XY} \\
\hat{\sigma}_{XY} & \hat{\sigma}^2_Y
\end{bmatrix} = \dfrac{1}{n-1} \sum_{i=1}^n \left( \begin{bmatrix} X_i & Y_i \end{bmatrix}^\top \begin{bmatrix} X_i & Y_i \end{bmatrix} \right)
\end{align*}

In [ ]:
hat_cov = np.zeros((2,2))
for i in range(n):
    hat_cov = hat_cov + (returns[i]-hat_mean).reshape((2,1))*(returns[i]-hat_mean).reshape((1,2))
hat_cov = hat_cov/(n-1)
hat_cov

In [ ]:
hat_varx = hat_cov[0,0]
hat_vary = hat_cov[1,1]
hat_covar = hat_cov[0,1]

#### Step 3: Estimate $\alpha^\star$ by:
\begin{align*}
\hat{\alpha}^\star = \dfrac{\hat{\sigma}^2_Y - \hat{\sigma}_{XY}}{\hat{\sigma}^2_X + \hat{\sigma}^2_Y - 2\hat{\sigma}_{XY}}.
\end{align*}

In [ ]:
round(optimal_alpha(hat_varx, hat_vary, hat_covar),3)

The real optimal strategy (if you really knew the distribution behind $(X, Y)$) would be $\alpha^\star = 0.6$. However, as you see $1,000$ samples, you can estimate $0.629$. Let's make a function that generalises this process.

In [ ]:
def generate_sample(n, varx=1, vary=1.25, covar=0.5): #Generate a sample of n returns
    mu = np.array([2 ,2])
    r = np.array([
            [  varx, covar],
            [ covar,  vary]])
    sample_returns = np.random.multivariate_normal(mu, r, size=n)
    return sample_returns
def hat_alpha(sample_returns): #Estimate the optimal investment strategy
    hat_mean = np.mean(sample_returns,0)
    hat_cov = np.zeros((2,2))
    n = np.size(sample_returns,0)
    for i in range(n):
        hat_cov = hat_cov + (sample_returns[i]-hat_mean).reshape((2,1))*(sample_returns[i]-hat_mean).reshape((1,2))
    hat_cov = hat_cov/(n-1)
    hat_varx = hat_cov[0,0]
    hat_vary = hat_cov[1,1]
    hat_covar = hat_cov[0,1]
    return round(optimal_alpha(hat_varx, hat_vary, hat_covar),3)

### Question 4: How does the estimation change with different samples? Can it be that the estimation $\hat{\alpha}^\star$ is very different from the real $\alpha$ (which you don't see), but the specific sample of returns has a huge bias?

In [ ]:
simulation = 1000 #Number of times to simulate
samples = 1000 #How many samples do you see in each 'scenario'?
estimations = np.zeros(simulation)
for sim in range(simulation):
    estimations[sim] = hat_alpha(generate_sample(samples))

In [ ]:
print("mean", round(np.mean(estimations),3),"min",np.min(estimations), "max", np.max(estimations))

**Answer 4:** As you can see, the mean of all estimations is equal to the true $\alpha$, which is the result of the unbiased estimation. However, the estimation varies between $[0.528, 0.686]$. How often do you think you receive 'bad' values? Let's see in a histogram:

In [ ]:
plt.hist(estimations,  bins = 20)  #Density=false would make counts
plt.axvline(x=0.6, color='r', linestyle='--', label = r"true value of $\alpha^\star$")
plt.legend()
plt.show()

Previously, you had $1,000$ simulations, and in each simulation, you had $1,000$ samples. In real life, you rarely get such a big sample. What if you just have $100$ samples? Let's review the histogram again.

In [ ]:
samples = 100 #How many samples do you see in each 'scenario'?
estimations = np.zeros(simulation)
for sim in range(simulation):
    estimations[sim] = hat_alpha(generate_sample(samples))
print("mean", round(np.mean(estimations),3),"min",np.min(estimations), "max", np.max(estimations))

In [ ]:
plt.hist(estimations,  bins = 30, color = 'g')  #Density=false would make counts
plt.axvline(x=0.6, color='r', linestyle='--', label = r"true value of $\alpha^\star$")
plt.legend()
plt.show()

### Question 5: In real life, you are in a single 'simulation'. As you don't know the real distribution of data, you cannot generate $1,000$ samples and decide accordingly. You need to take a risk and trust your $\hat{\alpha}^\star$ estimation. The question is, can you quantify this 'trust'?

**Answer 5:** It turns out, you can! Bootstrapping saves you here. Let's see the simple statistics of the previous simulation. 

First, denote the previous simulations by $B = 1,\ldots,1000$. Recall that in the very last experiment, in every simulation, you generated a sample of $100$ returns of $(X,Y)$. What is the mean of the estimations $\hat{\alpha}_b$ (where $b \in \{ 1,\ldots, B\}$ is the simulation number)?

You can get that through $\bar{\alpha} = \sum_{b=1}^B \hat{\alpha}_b$. Note that you need to drop the notation $\alpha^\star$, as it is clear that you are estimating the optimal $\alpha$.

In [ ]:
B = simulation #Notational convenience
bar_alpha = np.mean(estimations)
round(bar_alpha,3)

Now, let's compute the standard deviation of these estimates:

In [ ]:
standard_error = np.sqrt( np.sum(np.square(estimations - bar_alpha)) /(B-1))
round(standard_error,3)

You now conclude that the estimation satisfies $SE(\hat{\alpha}) \approx 0.081$ **based on the simulations**. So, if you randomly sample $100$ returns of $(X,Y)$ just once and estimate $\hat{\alpha}$ via the formula derived, i.e.

\begin{align*}
\hat{\alpha}^\star = \dfrac{\hat{\sigma}^2_Y - \hat{\sigma}_{XY}}{\hat{\sigma}^2_X + \hat{\sigma}^2_Y - 2\hat{\sigma}_{XY}}
\end{align*}

then you can expect this number to deviate from the *true optimal* $\alpha$, i.e.

\begin{align*}
\alpha^\star = \dfrac{\sigma^2_Y - \sigma_{XY}}{\sigma^2_X + \sigma^2_Y - 2\sigma_{XY}}
\end{align*}

by an amount of $0.082$ on average.

However, in the above experiments, you simulated a sample of returns (of size $100$) for $1,000$ times. Such sampling requires the distribution of $(X,Y)$. Again, in real life, you do not know the true distribution. You only have a single sample of $100$ returns of $(X, Y)$, and that is all you know. Let's denote the sample as $\mathcal{R} = \{(X_i, Y_i) \ : \ i = 1,\ldots,100 \}$.

So, how can you apply the previous simulation? 

Here comes bootstrapping. The idea is simple: as before, you simulate $1,000$ times, and in each simulation, you sample $100$ realisations of $(X,Y)$ returns. 

But this time, you use the returns $\mathcal{R}$ you already have. However, if you sample $100$ distinct values, this is basically equivalent to the true set of returns you have. 

Instead, here, you sample **with replacements**. Let's see how it works now.

#### Step 1: You are given a sample of $100$ returns, and you do not know the true distribution.

In [ ]:
given_returns = np.random.multivariate_normal(mu, r, size=n)

#### Step 2: Estimate the optimal investment strategy by estimating $\hat{\alpha}$ from the given returns.

In [ ]:
estimated_alpha = hat_alpha(given_returns)
estimated_alpha

#### Step 3: Observe the standard error associated with such an estimation technique by using bootstrapping.

In [ ]:
simulation = 1000#Same as before
B = simulation
samples = 100#Same
estimations = np.zeros(simulation)#Same as before
for sim in range(simulation):
    generated_sample = given_returns[np.random.randint(given_returns.shape[0], size=samples), :]
    estimations[sim] = hat_alpha(generated_sample)
print("mean", round(np.mean(estimations),3),"min",np.min(estimations), "max", np.max(estimations))

In [ ]:
#Compute standard error
bootstrap_error = np.sqrt( np.sum(np.square(estimations - np.mean(estimations))) /(B-1))
np.round(bootstrap_error,3)

In [ ]:
print("The estimation of the standard error from the simulations when we knew the distribution (which is very close to the truth due to the law of large numbers) was", round(standard_error,3))
print("While the bootstrap estimation of the standard error is", round(bootstrap_error,3))
print("Bootstrapping does not use any single information except for the given single 100-returns!")

### Question 6: When given a single sample of $100$ returns and estimate $\hat{\alpha}$, how can you obtain a confidence interval of the true $\alpha$?

**Answer 6:** This answer is inspired by ```Professor Martin Haugh```'s lecture notes on ```resampling methods```.
Let's review a $1-\beta$-confidence interval (CI) on the true $\alpha$. Fix any $\beta$.

In [ ]:
beta = 0.1

Let $q_l, \ q_u$ be the $\beta/2$ lower and upper quantiles of the bootstrap samples.

In [ ]:
ql = np.quantile(estimations, beta/2)
qu = np.quantile(estimations, 1- beta/2)

Then, the fraction of $\hat{\alpha}_b$ satisfying $q_l \leq \hat{\alpha}_b \leq q_u$ out of all $b=1,\ldots,B$ (where $B =1,000$ in this case) is $1-  \beta$. Let's observe this:

In [ ]:
collect = 0
for i in range(B):
    if estimations[i] <= qu and estimations[i] >= ql:
        collect += 1
collect/B

A naive approach is to claim $\alpha$ lies in the range $[q_l, q_u]$ which is a $1-\beta$ ($90\%$ in this case) confidence interval. Hence in our case (recall true $\alpha = 0.6$), the confidence interval is:

In [ ]:
np.array([ql, qu])

However, you can improve this. Recall that $\bar{\alpha}$ is the estimation by using the whole sample of 100 returns. It is:

In [ ]:
bar_alpha

The estimations that satisfy $q_l \leq \hat{\alpha}_b \leq q_u$ also satisfy $\bar{\alpha} - q_u \leq \bar{\alpha} - \hat{\alpha}_b \leq \bar{\alpha} -q_l$. Hence, $\bar{\alpha} - q_u$ and $\bar{\alpha}- q_l$ are lower/upper $\beta/2$ quantiles for $\bar{\alpha} - \hat{\alpha}_b$!

So adding $\bar{\alpha}$ to this equation, you will obtain: $2\bar{\alpha} - q_u \leq \alpha \leq 2\bar{\alpha}  - q_l$.

Thus, a $(1 - \beta)\%$ CI of the true $\alpha$ is $(2\bar{\alpha} - q_u, 2\bar{\alpha}  - q_l)$. In our case, a $90\%$ CI is:

In [ ]:
np.array([2*bar_alpha - qu, 2*bar_alpha - ql])

Note that previously, you observed that the minimum bootstrap estimate of the $\alpha$ was $0.35$ and the maximum was $0.85$, so you can observe how a $90\%$ CI is more compact.

### Answer this question: How would you modify the estimation of $\alpha$, the simulation and the bootstrapping for obtaining CI, if you were told that $(X,Y)$ has a normal distribution?

Hint: $$ \alpha = \frac{\sigma_X^2 + \sigma_Y^2 - 2 \rho \sigma_X \sigma_Y}{\sigma_Y^2 - \rho \sigma_X \sigma_Y} $$
 Use the above notation of $\alpha$, create the function and get the estimate.

In [ ]:
#Generate data

mu = [0, 0]
cov = [[1, 0.4],
       [0.4, 1]]

#Number of samples to generate
n_samples = 1000

#Generate bivariate normal data
data = np.random.multivariate_normal(mean=mu, cov=cov, size=n_samples)

#Split into X and Y components
X = data[:, 0]
Y = data[:, 1]

print("Generated Data:")
print("X:", X[:5])  # Print first 5 samples of X
print("Y:", Y[:5])  # Print first 5 samples of Y


In [ ]:

#Fit bivariate normal to data
mu = np.mean(data, axis=0)
cov = np.cov(data.T)
n_bootstrap = 10000

#Parametric bootstrap
alpha_bootstrap = []
for _ in range(n_bootstrap):
    sample = multivariate_normal.rvs(mean=mu, cov=cov, size=n)
    sigma_X = np.var(sample[:,0])
    sigma_Y = np.var(sample[:,1])
    rho = np.corrcoef(sample.T)[0,1]
    alpha = (sigma_Y**2 - rho * np.sqrt(sigma_X * sigma_Y)) / (sigma_X + sigma_Y - 2 * rho * np.sqrt(sigma_X * sigma_Y))
    alpha_bootstrap.append(alpha)

CI = np.percentile(alpha_bootstrap, [2.5, 97.5])
print("The confidence interval is given by ", CI)